# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [4]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一条延伸在绿草丛中的木板路。图中的环境呈现出郁郁葱葱的自然景观，天空是大片的蓝色，云彩轻盈洁白。木板路引导观者的视线向远方延伸，增加了图片的深度感和探索欲。四周高大的草地和远处的疏散树木构成了一个平和且宁静的乡村景致，适合散步和放松心情。整个场景显得非常和谐，透出一种宁静和平静的氛围。这样的景象可能位于一个公园或自然保护区内，是自然爱好者和摄影师的理想选择。', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图展现了一个美丽的自然风景。图片中的主要元素是一条木板步道，它贯穿于郁郁葱葱的绿色草地中。步道可能为游客或徒步者提供了一条明确的行走路径，避免踏入湿润的草地。\n\n图片背景是蓝天白云，带有一些轻柔的白云，增添了宁静和平和的氛围。四周的草地上散布着不同种类的矮树和野花，呈现出丰富的生态多样性。整个场景显得十分清新，似乎是一个适合放松身心、亲近自然的地方。光线明亮，色彩饱满，整体给人一种生机勃勃和宁静的感觉。这样的场所很适合散步、摄影或简单地享受大自然的美景。'

### 封装成一个函数 query_image_description

In [21]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(api_key='sk-fK3oicxkB7FWjB082fA4144c5fA3415cAf24D7C5714eC01d',base_url='https://api.bltcy.ai/v1')  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [14]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一种幽默的表现，通过将狗的头部与健美运动员的身体结合，创造了一种搞笑的对比效果。左侧的图片显示了一只“肌肉发达”的柴犬，体格非常壮观，模仿了健美运动员的形象。右侧则是一只正常体型的柴犬，坐着休息的样子。

图中的文字是对比这两种状态，“16岁的状，工作后的状”，暗示年轻时的活力和成熟后可能面临的身体变化。左侧标签包括“我就是一片云朵，身体印有希望，人们都喜欢大阳光，带来高人气和收视”，右侧的文本则是“好累啊，好想睡觉觉，走路带风不灵真心睡不醒，飙升我的小肚腩，啊我的腿啊腿啊。”




### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [27]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI(api_key='sk-fK3oicxkB7FWjB082fA4144c5fA3415cAf24D7C5714eC01d',base_url='https://api.bltcy.ai/v1')  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.bltcy.ai/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [28]:
content = query_base64_image_description("./images/gpt4v.jpg")
print(content)

这张图片上写的是中文。内容是关于某个技术会议或研究报告的信息，日期标记为“2024年7月24日”。文档的标题是“关于深圳大学校园价值背景研究”，下面列出了两个议题或关键点：

1. GPT-4研究报告
2. AI Translator研究报告

这表明该文件可能是会议的议程或是某个研究活动的计划概要，专注于GPT-4和AI翻译技术的研究。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [29]:
content = query_base64_image_description("./images/gpt4v.jpg")
print(content)

这张图片显示的是一张写有中文的笔记纸。文字内容可以翻译为：

“关于深化文化产业国际合作的建议
2024年7月22日24日
内容：
1. GPT-4应用分析
2. AI Translator的发展”

这是一份可能用于讨论或会议的提纲或笔记，主要讨论关于文化产业在全球化环境中如何通过使用高级技术如GPT-4和AI翻译工具来加深国际合作。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [30]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片显示的是一张写有中文的笔记纸。文字内容可以翻译为：

“关于深化文化产业国际合作的建议
2024年7月22日24日
内容：
1. GPT-4应用分析
2. AI Translator的发展”

这是一份可能用于讨论或会议的提纲或笔记，主要讨论关于文化产业在全球化环境中如何通过使用高级技术如GPT-4和AI翻译工具来加深国际合作。

![](./images/handwriting_1.jpg)

In [32]:
content = query_base64_image_description("./images/gpt4v.jpg")
display(Markdown(content))

这张图片显示的是一张手写的笔记，内容如下：

标题：研究深度网络安全应用可靠性途径
日期：2024年7月22日24日
内容：
1. GPT-4应用案例
2. AI Translator应用安全

这个笔记可能是关于使用深度网络技术，特别是GPT-4和AI翻译工具，在提高网络安全应用的可靠性中的案例研究或讨论。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。